#### **Loading required libraries**

In [1]:
!pip install textblob
!pip install wordcloud
!pip install kaggle
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Data Source


Here is the dataset link for the project

https://www.kaggle.com/datasets/nadyinky/sephora-products-and-skincare-reviews/code?select=reviews_0_250.csv

In [3]:
import os
from google.colab import files


In [4]:
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dibyapandey","key":"c9166557cd553af796010997024017a1"}'}

In [5]:
# Move the API key to the correct directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set permissions for the API key
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
# Download the dataset
!kaggle datasets download -d nadyinky/sephora-products-and-skincare-reviews

# Unzip the downloaded dataset
!unzip sephora-products-and-skincare-reviews.zip


 88% 129M/147M [00:02<00:00, 50.8MB/s]
100% 147M/147M [00:02<00:00, 58.5MB/s]
Archive:  sephora-products-and-skincare-reviews.zip
  inflating: product_info.csv        
  inflating: reviews_0-250.csv       
  inflating: reviews_1250-end.csv    
  inflating: reviews_250-500.csv     
  inflating: reviews_500-750.csv     
  inflating: reviews_750-1250.csv    


In [7]:
# Load the dataset
df_review1 = pd.read_csv('reviews_0-250.csv')
df_review2 = pd.read_csv('reviews_250-500.csv')
df_review3 = pd.read_csv('reviews_500-750.csv')
df_review4 = pd.read_csv('reviews_750-1250.csv')
df_review5 = pd.read_csv('reviews_1250-end.csv')
df_makeup_products = pd.read_csv('product_info.csv')

<ipython-input-7-bc696df2bd27>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_review1 = pd.read_csv('reviews_0-250.csv')
<ipython-input-7-bc696df2bd27>:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_review4 = pd.read_csv('reviews_750-1250.csv')
<ipython-input-7-bc696df2bd27>:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_review5 = pd.read_csv('reviews_1250-end.csv')


In [8]:
# Merge the dataframes into one
df_reviews = pd.concat([df_review1, df_review2, df_review3, df_review4, df_review5], ignore_index=True)

In [9]:
# Check for duplicates
duplicates = df_reviews.duplicated()


In [10]:
# Remove duplicates
df_reviews = df_reviews.drop_duplicates()


In [11]:
# Drop rows with missing values
df_reviews = df_reviews.dropna()

In [12]:
cols_to_use = list(set(df_makeup_products.columns) - set(df_reviews.columns))
cols_to_use.append('product_id')

print(cols_to_use)

['new', 'variation_value', 'variation_type', 'tertiary_category', 'brand_id', 'limited_edition', 'child_min_price', 'sephora_exclusive', 'ingredients', 'reviews', 'loves_count', 'sale_price_usd', 'out_of_stock', 'child_max_price', 'secondary_category', 'value_price_usd', 'online_only', 'highlights', 'primary_category', 'child_count', 'size', 'variation_desc', 'product_id']


In [13]:
# Perform the merge with an inner join on the 'product_id' column
result_df = pd.merge(df_makeup_products[cols_to_use], df_reviews, on='product_id', how='inner')


In [14]:
result_df['secondary_category'].unique()

array(['Moisturizers', 'Treatments', 'Eye Care', 'Lip Balms & Treatments',
       'Sunscreen', 'Cleansers', 'Value & Gift Sets', 'Masks',
       'Mini Size', 'Wellness', 'High Tech Tools', 'Self Tanners',
       'Shop by Concern'], dtype=object)

In [15]:
result_df

,new,variation_value,variation_type,tertiary_category,brand_id,limited_edition,child_min_price,sephora_exclusive,ingredients,reviews,...,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_name,brand_name,price_usd
0,0,2 oz/ 60 mL,Size,Moisturizers,6018,0,NaN,1,"['Collagen (Vegan)*, Water (Aqua, Eau), Ethylh...",1321.0,...,2023-02-09,"Ever since I bought this, I noticed that my sk...",Must have,medium,brown,oily,black,GENIUS Sleeping Collagen Moisturizer,Algenist,98.0
1,0,2 oz/ 60 mL,Size,Moisturizers,6018,0,NaN,1,"['Collagen (Vegan)*, Water (Aqua, Eau), Ethylh...",1321.0,...,2022-12-01,I’ve been using this for 2 months now and I ca...,Incredible Collagen Sleeping Moisturizer! Luxury!,light,brown,combination,blonde,GENIUS Sleeping Collagen Moisturizer,Algenist,98.0
2,0,2 oz/ 60 mL,Size,Moisturizers,6018,0,NaN,1,"['Collagen (Vegan)*, Water (Aqua, Eau), Ethylh...",1321.0,...,2022-05-27,"Great product. It is so moisturizing. However,...",Great but Jar isn’t filled all the way,lightMedium,green,dry,brown,GENIUS Sleeping Collagen Moisturizer,Algenist,98.0
3,0,2 oz/ 60 mL,Size,Moisturizers,6018,0,NaN,1,"['Collagen (Vegan)*, Water (Aqua, Eau), Ethylh...",1321.0,...,2022-05-11,I’m addicted! I’m on my 3rd jar and it’s becom...,Staple in my night routine!,mediumTan,brown,normal,brown,GENIUS Sleeping Collagen Moisturizer,Algenist,98.0
4,0,2 oz/ 60 mL,Size,Moisturizers,6018,0,NaN,1,"['Collagen (Vegan)*, Water (Aqua, Eau), Ethylh...",1321.0,...,2022-05-09,After I apply my serum I use this moisturizer ...,Moisturizing cream.,mediumTan,brown,combination,black,GENIUS Sleeping Collagen Moisturizer,Algenist,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294717,1,0.5 oz / 15 mL,Size,Eye Creams & Treatments,6169,0,NaN,1,"['Aqua/Water/Eau, Caprylic/Capric Triglyceride...",45.0,...,2023-03-06,The consistency is more of a lotion which I re...,Love!,mediumTan,brown,oily,brown,Peptides + C Energy Eye Concentrate with Vitam...,Youth To The People,48.0
294718,0,NaN,NaN,NaN,6169,1,NaN,1,"['Superfood Antioxidant Cleanser:', 'Water/Aqu...",8.0,...,2023-02-23,Purchased this when it was on sale for $255. G...,Amazing value,fair,brown,dry,brown,The Youth Vault: 13-Piece Vegan Skincare + App...,Youth To The People,340.0
294719,0,NaN,NaN,NaN,6169,1,NaN,1,"['Superfood Antioxidant Cleanser:', 'Water/Aqu...",8.0,...,2023-01-25,everything works sooo well and the merch is ni...,amazinggg,fairLight,hazel,combination,auburn,The Youth Vault: 13-Piece Vegan Skincare + App...,Youth To The People,340.0
294720,0,NaN,NaN,NaN,6169,1,NaN,0,"['Superfood Antioxidant Cleanser:', 'Water/Aqu...",4.0,...,2023-02-15,I really like the cleanser and the moisturizer...,Good Deal,lightMedium,brown,combination,auburn,Youth Stacks: Daily Skin Health Your Way for P...,Youth To The People,20.0


In [16]:
# Filter for moisturizers with a rating of 5
moisturizer_rating_5 = result_df[(result_df['secondary_category'].str.contains('moisturizer', case=False)) & (result_df['rating'] == 5)]
top_3_moisturizer_brands = moisturizer_rating_5['brand_name'].value_counts().nlargest(3).index.tolist()


# Display the moisturizer brands with a rating of 5
print("Moisturizer Brands with a rating of 5:")
print(top_3_moisturizer_brands)


Moisturizer Brands with a rating of 5:
['Tatcha', 'Drunk Elephant', 'Farmacy']


#### **Cosine Similarity**

In [33]:
data1 = open('/content/sample_data/tatcha.txt')
text1 = data1.read().replace('\n', '').replace('-', ',')
data2 = open('/content/sample_data/drunk_elephant.txt')
text2 = data2.read().replace('\n', '').replace('-', ',')
data3 = open('/content/sample_data/farmacy.txt')
text3 = data3.read().replace('\n', '').replace('-', ',')

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text_file_m, text_file_n):
    tfidf = vectorizer.fit_transform((text_file_m, text_file_n))
    return ((tfidf * tfidf.T).A)[0, 1]

print(cosine_sim(text1,text2))
print(cosine_sim(text2,text3))
print(cosine_sim(text1,text3))


0.5239151678462538
0.595662096899911
0.491527210203696
